Cython - оптимизация кода

Выполнили: Анастасия Смирнова, Арсений Беляев

В совместной курсовой была найдена функция get_notall_info, которую можно было оптимизировать.

In [19]:
import vk_api
from datetime import datetime
def get_notall_info(owner_id):
    
    titles = ['verified', 'subscribers', 'posts', 'views', 'likes',
          'comments', 'reposts']
    
    
    vk_session = vk_api.VkApi('ars-bel@mail.ru', '**')
    vk_session.auth()
    vk = vk_session.get_api()

    owner = '-' + str(owner_id)

    response1 = vk.wall.get(owner_id = owner,count = 1, extended = 1,
                                      fields = 'screen_name,verified,members_count,activity')['groups'][0]

    control_date = int(datetime.now().timestamp()) - 2592000

    post_date = int(datetime.now().timestamp())

    all_members = response1['members_count']
    all_posts = 0
    all_comments = 0
    all_likes = 0
    all_views = 0
    all_reposts = 0
    er_post = 0
    raiting = 0

    offset = 1

    while post_date > control_date:
        response = vk.wall.get(owner_id = owner, count = 100, offset = offset)['items']
        if len(response) != 0:
            for i in response:
                if int(i['date']) < int(control_date):

                    post_date = i['date']
                    break

                try:
                    all_comments += i['comments']['count']
                    all_likes += i['likes']['count']
                    all_views += i['views']['count']
                    all_reposts += i['reposts']['count']
                    all_posts +=1
                except:
                    pass

            offset += 100
        else:
            post_date = control_date


    info = [response1['verified'],
            all_members,all_posts, all_views, all_likes, all_comments, all_reposts]
    return info


Время до оптимизации:

In [20]:
%%timeit -n1 -r1 
get_notall_info(23148107) 

12.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%cython -a
import vk_api
from datetime import datetime
def get_notall_info(owner_id):
    
    titles = ['verified', 'subscribers', 'posts', 'views', 'likes',
          'comments', 'reposts']
    
    
    vk_session = vk_api.VkApi('ars-bel@mail.ru', '**')
    vk_session.auth()
    vk = vk_session.get_api()

    owner = '-' + str(owner_id)

    response1 = vk.wall.get(owner_id = owner,count = 1, extended = 1,
                                      fields = 'screen_name,verified,members_count,activity')['groups'][0]

    control_date = int(datetime.now().timestamp()) - 2592000

    post_date = int(datetime.now().timestamp())

    all_members = response1['members_count']
    all_posts = 0
    all_comments = 0
    all_likes = 0
    all_views = 0
    all_reposts = 0
    er_post = 0
    raiting = 0

    offset = 1

    while post_date > control_date:
        response = vk.wall.get(owner_id = owner, count = 100, offset = offset)['items']
        if len(response) != 0:
            for i in response:
                if int(i['date']) < int(control_date):

                    post_date = i['date']
                    break

                try:
                    all_comments += i['comments']['count']
                    all_likes += i['likes']['count']
                    all_views += i['views']['count']
                    all_reposts += i['reposts']['count']
                    all_posts +=1
                except:
                    pass

            offset += 100
        else:
            post_date = control_date


    info = [response1['verified'],
            all_members,all_posts, all_views, all_likes, all_comments, all_reposts]
    return info


Время после оптимизации:

In [21]:
%%timeit -n1 -r1 
get_notall_info(23148107)

10.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
